# Andrew Hennis BINF 760 Homework 1

## DNA melting point estimation using linear regression

At the top of my file, I'm importing all of my necessary packages;

- biobase to calculate gc content of sequences
- pandas to import the csv and for dataframes
- scikitlearn for their linear regression model and r2 score

In [ ]:
!pip install biobase

In [2]:
from biobase.analysis import Dna
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
df = pd.read_csv('Tm.csv')
df.head()

,sequence,Tm
0,GTGTTTATCCATCGAA,38.08
1,GACCTCGGCACTACAA,44.50
2,GGCTTCCGTGAGCTAC,46.50
3,TCTCATCGTAACGAGG,41.38
4,ATATAGTTGCTTGTAA,28.43


In [4]:
df["AT count"] = df["sequence"].str.count("A") + df["sequence"].str.count("T")
df["GC count"] = df["sequence"].str.count("G") + df["sequence"].str.count("C")
df.head()

,sequence,Tm,AT count,GC count
0,GTGTTTATCCATCGAA,38.08,10,6
1,GACCTCGGCACTACAA,44.50,7,9
2,GGCTTCCGTGAGCTAC,46.50,6,10
3,TCTCATCGTAACGAGG,41.38,8,8
4,ATATAGTTGCTTGTAA,28.43,12,4


In [5]:
df["GC Percentage"] = df["sequence"].apply(Dna.calculate_gc_content)
df["Inverse N"] = 1/df["sequence"].str.len()
df.head()

,sequence,Tm,AT count,GC count,GC Percentage,Inverse N
0,GTGTTTATCCATCGAA,38.08,10,6,37.50,0.0625
1,GACCTCGGCACTACAA,44.50,7,9,56.25,0.0625
2,GGCTTCCGTGAGCTAC,46.50,6,10,62.50,0.0625
3,TCTCATCGTAACGAGG,41.38,8,8,50.00,0.0625
4,ATATAGTTGCTTGTAA,28.43,12,4,25.00,0.0625


In [6]:
X_1 = df[["AT count", "GC count"]]
y = df["Tm"]

In [7]:
X_2 = df[["GC Percentage", "Inverse N"]]

### Model

In [8]:
model_no_intercept = LinearRegression(fit_intercept=False)
model_no_intercept.fit(X_1, y)

LinearRegression(fit_intercept=False)

In [9]:
model_intercept = LinearRegression()
model_intercept.fit(X_2, y)

LinearRegression()

### Results

In [10]:
# Look at model weights for model without y-intercept
print(f"intercept = {model_no_intercept.intercept_}")
print(f"coefficients = {model_no_intercept.coef_}")

intercept = 0.0
coefficients = [1.40982089 3.72170087]


In [11]:
# Look at model weights for model with y-intercept
print(f"intercept = {model_intercept.intercept_}")
print(f"coefficients = {model_intercept.coef_}")

intercept = 71.00366394944022
coefficients = [ 5.48958054e-01 -8.59317751e+02]


In [12]:
# Put coefficients together with independent variables
print("No intercept model")
for column, coefficient in zip(X_1.columns, model_no_intercept.coef_):
    print(column, coefficient)

No intercept model
AT count 1.409820887529343
GC count 3.7217008724750174


In [13]:
# Put coefficients together with independent variables
print("Intercept model")
for column, coefficient in zip(X_2.columns, model_intercept.coef_):
    print(column, coefficient)

Intercept model
GC Percentage 0.5489580535047246
Inverse N -859.3177508914174


### Question II

The intercept and coefficients of your fitted linear equations

---

Given this information, the final equations would be
1. $T_m(^\circ C) = 1.4098(AT) + 3.7217(GC)$
2. $T_m(^\circ C) = 71.0037 + 0.5489(\frac{(GC)}{N} \cdot 100) - 859.3177(\frac{1}{N})$

### Question III

A quantitative justification for which equation you believe best fits the data

---

To determine numerically which equation is a better predictor, I'm going see how each model scores. The `.score` method for linear regression is the `coefficient of determination` according to scikit learn. This coefficient has a best possible score of 1.0 (where the model perfectly predicts every true value) and a worst possible score of 0.0 (where the model makes the same prediction regardless of the input values). Therefore, the equation with the higher score is the equation that better fits the data.

In [14]:
r2_pred1 = model_no_intercept.score(X_1, y)
r2_pred1

0.892976507804992

In [15]:
r2_pred2 = model_intercept.score(X_2, y)
r2_pred2

0.9701361852268294

Since the model with the intercept has a near perfect score of 0.97, it is the clear winner.

### Question IV

a note indicating whether you think the parameters of your fits agree well with the two equations presented below

---

The original two equations are

1. $T_m(^\circ C)=2(AT)+4(CG)$
2. $T_m(^\circ C)=69.30+0.41(\frac{(GC)}{N} \cdot 100)-650(\frac{1}{N})$

The equations calculated from my models are

1. $T_m(^\circ C) = 1.41(AT) + 3.72(GC)$
2. $T_m(^\circ C) = 71.00 + 0.55(\frac{(GC)}{N} \cdot 100) - 859.32(\frac{1}{N})$

In [16]:
# Original equation 1
df["Tm original eq1"] = 2 * df["AT count"] + 4 * df["GC count"]

# Original equation 2
df["Tm original eq2"] = 69.30 + 0.41 * ((df["GC count"] / df["sequence"].str.len()) * 100) - 650 * (1 / df["sequence"].str.len())


In [17]:
df[["Tm", "Tm original eq1", "Tm original eq2"]]

,Tm,Tm original eq1,Tm original eq2
0,38.08,44,44.0500
1,44.50,50,51.7375
2,46.50,52,54.3000
3,41.38,48,49.1750
4,28.43,40,38.9250
...,...,...,...
299995,92.74,116,85.9000
299996,93.76,116,85.9000
299997,95.72,116,85.9000
299998,93.99,116,85.9000


In [18]:
r2_eq1 = r2_score(df["Tm"], df["Tm original eq1"], multioutput='uniform_average')
r2_eq2 = r2_score(df["Tm"], df["Tm original eq2"], multioutput='uniform_average')
print(f"R2 for equation 1:\nOriginal: {r2_eq1}\nPrediction: {r2_pred1}\n")
print(f"R2 for equation 2:\nOriginal: {r2_eq2}\nPrediction: {r2_pred2}")

R2 for equation 1:
Original: 0.5264653772775043
Prediction: 0.892976507804992

R2 for equation 2:
Original: 0.9065381573725817
Prediction: 0.9701361852268294


I think in general, the parameters of my fits agree with the parameters in the original equations. The discrepancy between the values for equation one might be due to the over estimation of the GC content's contribution to the melting temperature. The coefficient of 3.72 instead of 4 lowers this overestimation for every GC pair.